## Data Loaders

This notebook demonstrates the use of graph loader in `pyTigerGraph`. The job of a data loader is to pull data from the TigerGraph database. Currently, the following data loaders are provided:
* EdgeLoader, which returns batches of edges.
* VertexLoader, which returns batches of vertices.
* GraphLoader, which returns randomly sampled (probably disconnected) subgraphs in pandas `dataframe`, `PyG` or `DGL` format.
* NeighborLoader, which returns subgraphs using neighbor sampling in `dataframe`, `PyG` or `DGL` format.
* EdgeNeighborLoader, which returns subgraphs using neighbor sampling from edges in `dataframe`, `PyG` or `DGL` format.

Every data loader above can either get all the batches as a HTTP response (default) or stream every batch through Kafka. The former mechanism is good for testing with small graphs and it is fast, but it subjects to a data size limit of 2GB. For large graphs, the HTTP channel will likely fail due to size limit and network connectivity issues. Streaming via Kafka is offered for data robustness and scalability. Also, Kafka excels at multi-consumer use cases, and it is efficient for model search or hyperparameter tuning when there are multiiple consumers of the same data. 

The data loaders support both homogeneous and heterogenous graphs. By default, they load from all vertex and edge types and treat the graph as a homogeneous graph. But they also allow users to specify what vertex and edge types to load from and what attributes to load from each type. This way users will get heterogeneous graph outputs.

**Note**: For the data loaders to work, a UDF (User Defined Function) has to be installed into the TigerGraph database. It only needs to be installed once per database and will work for all data loaders and all graphs. 

### Connection to Database

The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. It is able to perform quite a few database tasks. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

**Note**: Secret instead of username/password is required for TG cloud DBs created after 7/5/2022. Otherwise, you can leave it blank.

In [1]:
from pyTigerGraph import TigerGraphConnection

In [2]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

<span style="color:red">Uncomment cell below and run to get and set token if token authentication is enabled</span>. 
* This is required for all databases on tgcloud.
* `<secret>` is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.
* If you don't know your secret, you can use `secret=conn.createSecret()` to create one.

In [ ]:
#conn.getToken(<secret>)

In [3]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'Paper': 2708}

In [4]:
# Number of edges for every type
conn.getEdgeCount()

{'Cite': 10556}

Uncomment below to install the UDF. It only needs to be installed once per database and will work for all data loaders and all graphs. **Note**: installing the UDF will overwrite any existing UDF in the database.

In [5]:
ExprFunctions="https://tg-mlworkbench.s3.us-west-1.amazonaws.com/udf/1.0/ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil=""  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

ExprFunctions installed succesfully


### Graph Loader

GraphLoader pulls (possibly disconnected) subgraphs from database. Different from NeighborLoader which produces connected subgraphs, this loader gets (random) batches of edges and vertices attached to those edges.

**Note**: For the first time you initialize the loader on a graph in TigerGraph, the initialization might take a minute as it installs the corresponding query to the database and optimizes it. However, the query installation only needs to be done once, so it will take no time when you initialize the loader on the same TG graph again.
        
There are two ways to use the data loader. See [here](https://github.com/TigerGraph-DevLabs/mlworkbench-docs/blob/main/tutorials/basics/2_dataloaders.ipynb) for examples.
* First, it can be used as an iterable, which means you can loop through it to get every batch of data. If you load all data at once (`num_batches=1`), there will be only one batch (of all the data) in the iterator.
* Second, you can access the `data` property of the class directly. If there is only one batch of data to load, it will give you the batch directly instead of an iterator, which might make more sense in that case. If there are multiple batches of data to load, it will return the loader itself.

Args:
* v_in_feats (list or dict, optional):
        Vertex attributes to be used as input features. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type.
        Only numeric and boolean attributes are allowed. The type of an attribute
        is automatically determined from the database schema. Defaults to None.
* v_out_labels (list or dict, optional):
        Vertex attributes to be used as labels for prediction. 
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type.
        Only numeric and boolean attributes are allowed. Defaults to None.
* v_extra_feats (list or dict, optional):
        Other attributes to get such as indicators of train/test data.
        If it is a list, then the attributes
        in the list from all vertex types will be selected. An error will be thrown if
        certain attribute doesn't exist in all vertex types. If it is a dict, keys of the 
        dict are vertex types to be selected, and values are lists of attributes to be 
        selected for each vertex type. 
        All types of attributes are allowed. Defaults to None.
* e_in_feats (list or dict, optional):
        Edge attributes to be used as input features. 
        If it is a list, then the attributes
        in the list from all edge types will be selected. An error will be thrown if
        certain attribute doesn't exist in all edge types. If it is a dict, keys of the 
        dict are edge types to be selected, and values are lists of attributes to be 
        selected for each edge type.
        Only numeric and boolean attributes are allowed. The type of an attribute
        is automatically determined from the database schema. Defaults to None.
* e_out_labels (list or dict, optional):
        Edge attributes to be used as labels for prediction. 
        If it is a list, then the attributes in the list from all edge types will be 
        selected. An error will be thrown if certain attribute doesn't exist in all 
        edge types. If it is a dict, keys of the dict are edge types to be selected, 
        and values are lists of attributes to be selected for each edge type.
        Only numeric and boolean attributes are allowed. Defaults to None.
* e_extra_feats (list or dict, optional):
        Other edge attributes to get such as indicators of train/test data. 
        If it is a list, then the attributes in the list from all edge types will be 
        selected. An error will be thrown if certain attribute doesn't exist in all 
        edge types. If it is a dict, keys of the dict are edge types to be selected, 
        and values are lists of attributes to be selected for each edge type.
        All types of attributes are allowed. Defaults to None.
* batch_size (int, optional):
        Number of edges in each batch.
        Defaults to None.
* num_batches (int, optional):
        Number of batches to split the edges.
        Defaults to 1.
* shuffle (bool, optional):
        Whether to shuffle the data before loading.
        Defaults to False.
* filter_by (str, optional):
        A boolean attribute used to indicate which edges can be included.
        Defaults to None.
* output_format (str, optional):
        Format of the output data of the loader.
        Only "PyG", "DGL" and "dataframe" are supported. Defaults to "dataframe".
* add_self_loop (bool, optional):
        Whether to add self-loops to the graph. Defaults to False.
* loader_id (str, optional):
        An identifier of the loader which can be any string. It is
        also used as the Kafka topic name. If `None`, a random string will be generated
        for it. Defaults to None.
* buffer_size (int, optional):
        Number of data batches to prefetch and store in memory. Defaults to 4.
* kafka_address (str, optional):
        Address of the kafka broker. Defaults to None.
* kafka_max_msg_size (int, optional):
        Maximum size of a Kafka message in bytes.
        Defaults to 104857600.
* kafka_num_partitions (int, optional):
        Number of partitions for the topic created by this loader.
        Defaults to 1.
* kafka_replica_factor (int, optional):
        Number of replications for the topic created by this
        loader. Defaults to 1.
* kafka_retention_ms (int, optional):
        Retention time for messages in the topic created by this
        loader in milliseconds. Defaults to 60000.
* kafka_auto_del_topic (bool, optional):
        Whether to delete the Kafka topic once the
        loader finishes pulling data. Defaults to True.
* kafka_address_consumer (str, optional):
        Address of the kafka broker that a consumer
        should use. Defaults to be the same as `kafkaAddress`.
* kafka_address_producer (str, optional):
        Address of the kafka broker that a producer
        should use. Defaults to be the same as `kafkaAddress`.
* timeout (int, optional):
        Timeout value for GSQL queries, in ms. Defaults to 300000.

#### Load the whole graph directly to local as a PyG graph

In [6]:
%%time
graph_loader = conn.gds.graphLoader(
    num_batches=1,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG")

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 220 ms, sys: 58.1 ms, total: 278 ms
Wall time: 49.9 s


In [7]:
%%time
# Get the only batch of data via the `data` property
data = graph_loader.data

CPU times: user 1.71 s, sys: 176 ms, total: 1.89 s
Wall time: 1.7 s


In [8]:
data

Data(edge_index=[2, 10556], edge_feat=[10556], is_train=[10556], is_val=[10556], x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

#### Get subgraphs through http

In [9]:
%%time
graph_loader = conn.gds.graphLoader(
    num_batches=10,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=True,
    filter_by=None
)

CPU times: user 3.08 ms, sys: 1.99 ms, total: 5.07 ms
Wall time: 7.76 ms


In [10]:
%%time
for i, batch in enumerate(graph_loader):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
Data(edge_index=[2, 938], edge_feat=[938], is_train=[938], is_val=[938], x=[954, 1433], y=[954], train_mask=[954], val_mask=[954], test_mask=[954])
----Batch 1----
Data(edge_index=[2, 1089], edge_feat=[1089], is_train=[1089], is_val=[1089], x=[1260, 1433], y=[1260], train_mask=[1260], val_mask=[1260], test_mask=[1260])
----Batch 2----
Data(edge_index=[2, 1086], edge_feat=[1086], is_train=[1086], is_val=[1086], x=[1237, 1433], y=[1237], train_mask=[1237], val_mask=[1237], test_mask=[1237])
----Batch 3----
Data(edge_index=[2, 1062], edge_feat=[1062], is_train=[1062], is_val=[1062], x=[1256, 1433], y=[1256], train_mask=[1256], val_mask=[1256], test_mask=[1256])
----Batch 4----
Data(edge_index=[2, 1059], edge_feat=[1059], is_train=[1059], is_val=[1059], x=[1149, 1433], y=[1149], train_mask=[1149], val_mask=[1149], test_mask=[1149])
----Batch 5----
Data(edge_index=[2, 1020], edge_feat=[1020], is_train=[1020], is_val=[1020], x=[997, 1433], y=[997], train_mask=[997], val_mask=

#### For heterogeneous graphs

Since `Cora` is a homogeneous graph, we will connect to a different graph to demostrate the use case of heterogeneous graphs.

In [11]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="hetero",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [12]:
print(conn.gsql("ls"))

---- Graph hetero
Vertex Types:
- VERTEX v0(PRIMARY_ID id INT, x LIST<DOUBLE>, y INT, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v1(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
- VERTEX v2(PRIMARY_ID id INT, x LIST<DOUBLE>, train_mask BOOL, val_mask BOOL, test_mask BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
Edge Types:
- DIRECTED EDGE v0v0(FROM v0, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v1(FROM v1, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v1v2(FROM v1, TO v2, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v0(FROM v2, TO v0, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v1(FROM v2, TO v1, is_train BOOL, is_val BOOL)
- DIRECTED EDGE v2v2(FROM v2, TO v2, is_train BOOL, is_val BOOL)

Graphs:
- Graph hetero(v0:v, v1:v, v2:v, v0v0:e, v1v1:e, v1v2:e, v2v0:e,

In [13]:
loader = conn.gds.graphLoader(
    v_in_feats={"v0": ["x"],
                "v1": ["x"]},
    v_out_labels={"v0": ["y"]},
    v_extra_feats={"v0": ["train_mask", "val_mask", "test_mask"]},
    e_extra_feats={"v0v0": ["is_train", "is_val"],
                    "v1v1": ["is_train", "is_val"]},
    batch_size=1024,
    shuffle=False,
    filter_by=None,
    output_format="PyG",
    add_self_loop=False,
    loader_id=None,
    buffer_size=4
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.


In [14]:
%%time
for i, batch in enumerate(loader):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
HeteroData(
  v0={
    x=[76, 77],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[110, 57] },
  (v0, v0v0, v0)={
    edge_index=[2, 413],
    is_train=[413],
    is_val=[413]
  },
  (v1, v1v1, v1)={
    edge_index=[2, 558],
    is_train=[558],
    is_val=[558]
  }
)
----Batch 1----
HeteroData(
  v0={
    x=[76, 77],
    y=[76],
    train_mask=[76],
    val_mask=[76],
    test_mask=[76]
  },
  v1={ x=[110, 57] },
  (v0, v0v0, v0)={
    edge_index=[2, 297],
    is_train=[297],
    is_val=[297]
  },
  (v1, v1v1, v1)={
    edge_index=[2, 486],
    is_train=[486],
    is_val=[486]
  }
)
CPU times: user 70.4 ms, sys: 11 ms, total: 81.4 ms
Wall time: 98.4 ms


### Streaming through Kafka

**Note**: Kafka streaming is only available for the Enterprise Edition. You need to install the UDF with Kafka support.

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph",
    gsqlSecret="" # secret instead of user/pass is required for TG cloud DBs created after 7/5/2022  
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [ ]:
ExprFunctions="./ExprFunctions.hpp"  # For enterprise users, please use the link you received.
ExprUtil="./ExprUtil.hpp"  # For enterprise users, please use the link you received.
conn.installUDF(ExprFunctions, ExprUtil)

#### Configure Kafka
Set up Kafka here. Once configured, the settings will be shared with all newly created data loaders and no need to set up Kafka for each loader. Please see official https://docs.tigergraph.com/pytigergraph/current/gds/gds for detailed settings.

In [ ]:
conn.gds.configureKafka(
    kafka_address="127.0.0.1:9092",
    kafka_security_protocol="SASL_PLAINTEXT",
    kafka_sasl_mechanism="PLAIN",
    kafka_sasl_plain_username="your username",
    kafka_sasl_plain_password="your password"
)

#### Get subgraphs

In [11]:
%%time
graph_loader = conn.gds.graphLoader(
    num_batches=10,
    v_in_feats = ["x"],
    v_out_labels = ["y"],
    v_extra_feats = ["train_mask", "val_mask", "test_mask"],
    e_in_feats=["time"],
    e_out_labels=[],
    e_extra_feats=["is_train", "is_val"],
    output_format = "PyG",
    shuffle=True,
    filter_by=None,
    kafka_address="127.0.0.1:9092"
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.
CPU times: user 55.7 ms, sys: 15.1 ms, total: 70.8 ms
Wall time: 26 s


In [12]:
%%time
for i, batch in enumerate(graph_loader):
    print("----Batch {}----".format(i))
    print(batch)

----Batch 0----
Data(edge_index=[2, 1018], edge_feat=[1018], is_train=[1018], is_val=[1018], x=[1006, 1433], y=[1006], train_mask=[1006], val_mask=[1006], test_mask=[1006])
----Batch 1----
Data(edge_index=[2, 1036], edge_feat=[1036], is_train=[1036], is_val=[1036], x=[1226, 1433], y=[1226], train_mask=[1226], val_mask=[1226], test_mask=[1226])
----Batch 2----
Data(edge_index=[2, 1083], edge_feat=[1083], is_train=[1083], is_val=[1083], x=[1234, 1433], y=[1234], train_mask=[1234], val_mask=[1234], test_mask=[1234])
----Batch 3----
Data(edge_index=[2, 1074], edge_feat=[1074], is_train=[1074], is_val=[1074], x=[1263, 1433], y=[1263], train_mask=[1263], val_mask=[1263], test_mask=[1263])
----Batch 4----
Data(edge_index=[2, 1023], edge_feat=[1023], is_train=[1023], is_val=[1023], x=[1107, 1433], y=[1107], train_mask=[1107], val_mask=[1107], test_mask=[1107])
----Batch 5----
Data(edge_index=[2, 1072], edge_feat=[1072], is_train=[1072], is_val=[1072], x=[1040, 1433], y=[1040], train_mask=[1040